# Bayesian Inference

Examples of Bayesian Inference for join ILAAPT/INAAPT Spring Meeting April 24, 2021

The `empiricaldist` package is from [Allen B Downey](https://github.com/AllenDowney)


In [ ]:
# import some packages we'll need later

import pymc3 as pm  # Markov Chain Monte Carlo
import arviz as az  # visualization
import numpy as np  # numpy
import matplotlib.pyplot as plt # make pretty graphs
import pandas as pd
from empiricaldist import Pmf   # Learn about Bayesian inference. A "Pmf" is a "probability mass function"


In [ ]:
# start with a basket of fruit!
A=Pmf({'orange':2, 'apple':5, 'plum':7});
A

In [ ]:
# convert counts to probabilities
A.normalize()
A

In [ ]:
# it's easy to retrieve the probability of an outcome
A['apple']

In [ ]:
# We can multiply probabilties by a simple list of numbers and it "does the right thing" for Bayesian work.
A*[1,2,3]

In [ ]:
B=Pmf({'orange':4, 'apple':1, 'plum':12}); 
B.normalize()
B

In [ ]:
A.choice()

In [ ]:
B.choice()

In [ ]:
# a nice way to display both baskets together
pd.DataFrame({'A':A, 'B':B})

In [ ]:
hypotheses = Pmf(1,['A','B'])
hypotheses.normalize()
hypotheses

In [ ]:
data = 'orange'
A[data],B[data]

In [ ]:
hypotheses = hypotheses*[A[data],B[data]]
hypotheses.normalize()
hypotheses

In [ ]:
data = 'plum'
hypotheses = hypotheses*[A[data],B[data]]
hypotheses.normalize()
hypotheses

In [ ]:
data = 'plum'
hypotheses = hypotheses*[A[data],B[data]]
hypotheses.normalize()
hypotheses

In [ ]:
for data in B.choice(10):
    hypotheses = hypotheses*[A[data],B[data]]

hypotheses.normalize()
hypotheses    

# Machinery to handle continuous distributions

Many times we don't have discrete distributions, but rather continuous random variables. We can fake this with PMFs, but to really get it right, we need something a bit more sophisticated. Here is a simple example of a linear fit using randomly generated data to illustrate the pymc3 machinery.

In [ ]:
# generative model, simple normal distribution

x1 = np.random.normal(size=20)*.5 + 2

with pm.Model() as normal_model:
    mu = pm.Normal('mu',mu=0,sigma=5)  # prior distribution for mu
    sig = pm.HalfNormal('sig',sigma=2) # prior distribution for sigma
    
    x_obs = pm.Normal("x_obs", mu=mu, sigma=sig, observed=x1) # relationship to observed data
    trace = pm.sample(1000, return_inferencedata=True)
    
print(az.summary(trace, kind="stats"))
_=az.plot_trace(trace)

In [ ]:
x1 = np.random.normal(size=100)*.3+4.1
x2 = np.random.normal(size=70)*.2+4.3

_=plt.hist(x1)
_=plt.hist(x2)

In [ ]:
with pm.Model() as diff_model:
    mu1 = pm.Uniform('mu1',0,10)  # prior distribution for mu1
    mu2 = pm.Uniform('mu2',0,10)  # prior for mu2
    sig1 = pm.HalfNormal('sig1',sigma=2) # prior distribution for sigma1
    sig2 = pm.HalfNormal('sig2',sigma=2) # prior distribution for sigma2
    
    x1_obs = pm.Normal("x1_obs", mu=mu1, sigma=sig1, observed=x1) # relationship to observed data
    x2_obs = pm.Normal("x2_obs", mu=mu2, sigma=sig2, observed=x2) # relationship to observed data

    diff = pm.Deterministic("x_diff", mu2 - mu1)

    trace = pm.sample(1000, return_inferencedata=True)
    
print(az.summary(trace, kind="stats"))
_=az.plot_trace(trace)

In [ ]:
x = np.linspace(0,10,20)
m_gen = 2.5
b_gen = 1.3
s_gen = 0.3

y_th = m_gen*x + b_gen
y_noise = y_th + s_gen*np.random.normal(size=(len(x))) # y = m*x + b + noise

plt.plot(x,y_noise,'b.',label='fake data (w/noise)')
plt.plot(x,y_th,'r-',label='theory: no noise')
plt.title('generated test data')
plt.grid()
plt.legend()

In [ ]:
with pm.Model() as model:
    m = pm.Uniform('m',0,5)  # prior distribution for m
    b = pm.Uniform('b',0,5)  # prior distribution for b
    s = pm.Uniform('s',0,2)  # prior distribution for s
    
    y_obs = pm.Normal('y_obs',mu=m*x+b, sigma=s, observed=y_noise) # relate to measured values


In [ ]:
with model:
    trace = pm.sample(1000, return_inferencedata=True)
    
print(az.summary(trace, kind="stats"))
_=az.plot_trace(trace)

In [ ]:
import matplotlib

print("pymc3 is version {:s}".format(pm.__version__))
print("arviz is version {:s}".format(az.__version__))
print("numpy is version {:s}".format(np.__version__))
print("pandas is version {:s}".format(pd.__version__))
print("matplotlib is version {:s}".format(matplotlib.__version__))